In [ ]:
from gurobipy import *
from passenger import Passenger
import pandas as pd

In [ ]:
# Load passengers

data_seating = pd.ExcelFile(r'DataSeating.xlsx')
flight_dates = data_seating.sheet_names

flight_data = data_seating.parse(sheet_name = flight_dates[0])
flight_data.drop(flight_data.tail(2).index, inplace=True)
flight_data = flight_data.fillna(0)

In [ ]:
for group in flight_data.iterrows():
    people = group[1][["Femmes","Hommes","Enfants", "WCHR", "WCHB"]]
    
    for p in people.index:
        print(p,people[p],"\n")

In [105]:
passengers = []
passengers.append(Passenger(1,1,"men","Economic"))
passengers.append(Passenger(1,2,"men","Economic"))
passengers.append(Passenger(1,3,"women","Economic"))
passengers.append(Passenger(2,0,"women","Economic"))
passengers.append(Passenger(2,4,"women","Economic"))
passengers.append(Passenger(2,5,"women","Economic"))

width = 6
height = 28
X = [-3, -2, 1, 1, 2, 3]
Y = list(range(-14,0)) + list(range(1,14+1))
Y

total_weight = sum([p.weight for p in passengers])


In [115]:
m = Model("model_v1")

allocation = m.addVars(len(passengers), width, height, vtype=GRB.BINARY, name="Allocation")

# Make sure that there is at most one person per seat
C_limited_seat = m.addConstrs((sum([allocation[p.passenger_id,i,j] for p in passengers]) <= 1 for i in range(width) for j in range(height)))

# Maker sure that everyone is seated and only in one spot
C_serve_all = m.addConstrs((sum([allocation[p.passenger_id,i,j] for i in range(width) for j in range(height) ]) == 1 for p in passengers))

# Make sure that the center of mass is centralized
C_bary_X_p = m.addConstr(sum([p.weight * allocation[p.passenger_id,i,j]*X[i] for p in passengers for i in range(width) for j in range(height)])/total_weight <= 1)
C_bary_X_m = m.addConstr(sum([p.weight * allocation[p.passenger_id,i,j]*X[i] for p in passengers for i in range(width) for j in range(height)])/total_weight >= -1)
C_bary_Y_p = m.addConstr(sum([p.weight * allocation[p.passenger_id,i,j]*Y[j] for p in passengers for i in range(width) for j in range(height)])/total_weight <= 2)
C_bary_Y_m = m.addConstr(sum([p.weight * allocation[p.passenger_id,i,j]*Y[j] for p in passengers for i in range(width) for j in range(height)])/total_weight >= -2)


m.update()

m.setObjective(allocation.sum(), GRB.MAXIMIZE)

m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 178 rows, 1008 columns and 6048 nonzeros
Model fingerprint: 0x1316a275
Variable types: 0 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 6.0000000
Presolve added 4 rows and 4 columns
Presolve time: 0.02s
Presolved: 182 rows, 1012 columns, 3672 nonzeros
Variable types: 1 continuous, 1011 integer (1008 binary)

Explored 0 nodes (0 simplex iterations) in 0.04 seconds (0.02 work units)
Thread count was 12 (of 12 available processors)

Solution count 1: 6 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e+00, best bound 6.000000000000e+00, gap 0.0000%


In [116]:
for a in allocation:
    if allocation[a].x == 1:
        print(a[0], X[a[1]], Y[a[2]])


0 3 6
1 2 -9
2 1 5
3 -2 7
4 -3 -7
5 -3 -14
